In [ ]:
import ipywidgets as widgets
from IPython.display import display

import pandas as pd
from glob import glob
import re
import os

def get_images_from_id(pid, data_dir=r'./data/CDD-CESM/Low energy images of CDD-CESM/', prefix='P'):
    image_glob = prefix +  "_".join(re.findall(r'(\d+)([L,R]+)', pid)[0]) + "*.jpg"
    image_glob = os.path.join(data_dir, image_glob)
    return glob(image_glob)

#Extract
all_results = pd.read_csv('output/all_results.csv', index_col=1)
ensemble_predictions = pd.read_csv('output/ensemble_predictions.csv')
annotation = pd.read_csv('data/annotation/annotation.csv', index_col=-1)[['symptoms']]

#Transform
all_results = all_results.merge(annotation, left_index=True, right_index=True, how='left')
all_results['images'] = [get_images_from_id(pid) for pid in all_results.index]

#Load
DATA_DF = all_results.copy()


pid_select = widgets.SelectMultiple(
    options=DATA_DF.index,
    description='Patient ID:',
    disabled=False,
)


def create_text_area(pids):
    return [    widgets.Textarea(
            value=DATA_DF.loc[pid]['symptoms'],
            placeholder='Type something',
            description='Symptoms:',
            disabled=False
        ) for pid in pids]


output = widgets.Accordion(children=[])

def update_display(b):
    pids = list(b.owner.value)
    text = DATA_DF.loc[pids]['symptoms']
    output.children = create_text_area(pids)
    for i, pid in enumerate(pids):
        output.set_title(i, pid)

display(pid_select, output)

pid_select.observe(update_display, names=['value'], type='change')